# Sample data from the Choroid plexus
Data comes from [GEO (GSM3687214)](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSM3687214), and the data are included the the default adobo installation.

First we need to load the Python package:

In [1]:
import adobo as ad

adobo version 0.2.38. Documentation: https://oscar-franzen.github.io/adobo/


### Load the gene expression data
The parameter `bundled=True` makes adobo aware that this dataset is sample data and the user does not need to worry about paths. The `desc` parameter can be used to set a description of the dataset, can be a long description or a mnemonic. The `verbose=True` can be omitted, but instructs adobo to be more talkative.

In [2]:
exp = ad.IO.load_from_file('GSM3687214_countTable_K11_choroidPlexus_wt.txt.gz',
                           output_file='GSM3687214.joblib',
                           verbose=True,
                           desc='Sample data (GSM3687214; Choroid plexus)',
                           bundled=True)

Using a sparse matrix structure, please wait
Generating cell summary statistics...
Memory usage of loaded data: 83.51 MB
27,998 genes and 4,055 cells were loaded
loading took 0.2 minutes
